In [112]:
import tensorflow as tf
tf.__version__
tf.config.experimental.enable_op_determinism()
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
  tf.config.experimental.set_memory_growth(gpu, True)

In [113]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import MinMaxScaler
import random

In [114]:
tf.keras.backend.clear_session()

In [115]:
subject = 'Minas Gerais - Consumo de Cimento (t)'
start_index = 0
split_index = 191 #Referente aos 230 anos de input 
window_size = 36
train_split = split_index + 1 - 30

In [116]:
data = pd.read_csv('2003_mo_model_input_MG.csv')
data = data[[col for col in data.columns if col != subject] + [subject]] #Seta consumo (target) para a coluna final
data =data.drop([' NFSP - Fluxo Mensal (Milhões de reais)'], axis=1)
data

,Unnamed: 0,Minas Gerais - IDH,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Minas Gerais - value,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Minas Gerais - Produção de Cimento (t),Minas Gerais - Desemprego,Minas Gerais - PIB - Estadual,Minas Gerais - PIB - Construção Civil,Minas Gerais - PIB - Per Capita,Minas Gerais - PIB - Preços de Mercado,Minas Gerais - Consumo de Cimento (t)
0,2003-1,0.756537,0.724032,11.520143,1.611498,1.036534,1.772069e+08,7.330309e+06,0.969649,0.334536,1.881651,889.396645,1304.196449,18.796583,16.168039,85.634952,0.300000,648.891160,8.242462,2.937752e+08,1.725077e+07,14.096054,2.759117e+08,247.134
1,2003-2,0.756708,0.690297,11.189862,1.450389,0.993449,1.773884e+08,7.335910e+06,0.950783,0.335670,1.872777,889.450610,1289.394416,18.293640,15.115445,83.129710,0.300000,652.396314,8.236470,2.941480e+08,1.726959e+07,14.102603,2.760648e+08,305.708
2,2003-3,0.756879,0.669681,10.820792,1.870184,0.973020,1.775699e+08,7.341511e+06,0.938332,0.336553,1.638997,889.584406,1285.306737,18.099369,14.148246,79.655520,0.300000,655.911573,8.230479,2.945208e+08,1.728841e+07,14.109152,2.762178e+08,284.168
3,2003-4,0.757050,0.660494,10.417840,1.355287,0.940489,1.777514e+08,7.347112e+06,0.926401,0.336531,1.339260,890.048010,1258.400433,17.617245,13.127518,77.295209,0.380914,656.746430,8.224488,2.948935e+08,1.730723e+07,14.115701,2.763709e+08,286.594
4,2003-5,0.757222,0.648337,9.959690,1.585538,0.917493,1.779329e+08,7.352713e+06,0.951683,0.336406,1.204123,890.232394,1261.365893,17.005052,12.074203,74.991327,0.574731,660.114672,8.218497,2.952663e+08,1.732605e+07,14.122250,2.765239e+08,312.455
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,2022-8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.622619,2.062026,889.187698,945.152793,17.240415,12.088686,75.233138,2.017803,1250.988075,NaN,NaN,NaN,NaN,NaN,633.858
236,2022-9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.621866,2.251095,888.366445,1018.311245,17.761732,13.148325,77.577952,2.015372,1247.030458,NaN,NaN,NaN,NaN,NaN,612.428
237,2022-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.620866,2.453236,887.969082,1073.369131,18.062904,13.776904,78.876317,2.004010,1244.634022,NaN,NaN,NaN,NaN,NaN,579.831
238,2022-11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.619585,2.541913,887.580305,1060.732040,17.948706,13.980504,80.222356,2.039412,1241.214294,NaN,NaN,NaN,NaN,NaN,559.950


In [117]:
input_data = data.iloc[:split_index + 1,1:-1]
mean = np.mean(input_data, axis=0)
stddev =  np.std(input_data, axis=0)
input_data = ((input_data - mean) /stddev)
# scaler=MinMaxScaler()
# scaled_data = scaler.fit_transform(input_data)
# input_data = pd.DataFrame(scaled_data, columns=input_data.columns)
input_data

,Minas Gerais - IDH,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Minas Gerais - value,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Minas Gerais - Produção de Cimento (t),Minas Gerais - Desemprego,Minas Gerais - PIB - Estadual,Minas Gerais - PIB - Construção Civil,Minas Gerais - PIB - Per Capita,Minas Gerais - PIB - Preços de Mercado
0,-1.917914,2.723741,4.398348,2.132459,3.890153,-2.042341,-2.389042,3.122582,-1.524203,0.569744,0.405068,0.701907,0.185208,1.501626,2.473320,-4.333238,-2.146709,-0.699400,-1.745399,-1.973963,-3.018821,-2.686146
1,-1.892160,2.350880,4.222509,1.592202,3.551840,-2.014760,-2.352139,2.970356,-1.508666,0.548062,0.454590,0.535687,-0.282236,0.739319,1.716824,-4.333238,-2.125900,-0.706157,-1.725016,-1.923436,-2.934320,-2.623049
2,-1.866406,2.123016,4.026019,2.999926,3.391423,-1.987179,-2.315236,2.869895,-1.496577,-0.023148,0.577370,0.489784,-0.462795,0.038856,0.667739,-4.333238,-2.105031,-0.712914,-1.704633,-1.872910,-2.849818,-2.559951
3,-1.840653,2.021477,3.811492,1.273293,3.135979,-1.959598,-2.278333,2.773628,-1.496879,-0.755514,1.002803,0.187639,-0.910888,-0.700373,-0.044993,-4.152822,-2.100075,-0.719672,-1.684250,-1.822384,-2.765317,-2.496853
4,-1.814899,1.887113,3.567576,2.045407,2.955412,-1.932017,-2.241431,2.977624,-1.498597,-1.085703,1.172006,0.220940,-1.479870,-1.463202,-0.740685,-3.720662,-2.080078,-0.726429,-1.663867,-1.771858,-2.680816,-2.433756
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,1.315321,-2.010387,-0.572934,-1.311366,0.589021,1.365911,0.389193,-1.749976,0.874645,0.970821,0.258320,-0.805508,-0.875907,-0.534471,0.236522,-0.587203,0.774558,0.789659,1.149269,-0.772657,0.183801,0.694771
188,1.308842,-1.870713,-0.588777,-1.417143,1.043728,1.376610,0.370392,-1.593005,0.890157,1.225461,-0.397241,-0.305413,-0.235972,0.004219,0.306158,-0.620933,0.806353,0.784605,1.140354,-0.757425,0.173158,0.684584
189,1.302363,-1.806230,-0.612606,-1.597784,1.387010,1.387308,0.351592,-1.351489,0.906034,1.709580,-0.508089,0.370290,0.502599,0.515241,0.206625,-0.975326,0.844284,0.779552,1.131439,-0.742194,0.162516,0.674397
190,1.295885,-1.727496,-0.640956,-1.400787,1.815728,1.398006,0.332791,-1.198492,0.919415,1.790915,-0.713258,0.845625,1.058426,0.868770,0.089936,-1.671516,0.884141,0.774498,1.122523,-0.726963,0.151874,0.664209


In [118]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-12)
target_data

0      249.680
1      219.561
2      304.641
3      262.564
4      299.209
        ...   
235        NaN
236        NaN
237        NaN
238        NaN
239        NaN
Name: Minas Gerais - Consumo de Cimento (t), Length: 240, dtype: float64

In [119]:
# input para treinamento
train_input = input_data.iloc[start_index:train_split]
train_input

,Minas Gerais - IDH,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Minas Gerais - value,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Minas Gerais - Produção de Cimento (t),Minas Gerais - Desemprego,Minas Gerais - PIB - Estadual,Minas Gerais - PIB - Construção Civil,Minas Gerais - PIB - Per Capita,Minas Gerais - PIB - Preços de Mercado
0,-1.917914,2.723741,4.398348,2.132459,3.890153,-2.042341,-2.389042,3.122582,-1.524203,0.569744,0.405068,0.701907,0.185208,1.501626,2.473320,-4.333238,-2.146709,-0.699400,-1.745399,-1.973963,-3.018821,-2.686146
1,-1.892160,2.350880,4.222509,1.592202,3.551840,-2.014760,-2.352139,2.970356,-1.508666,0.548062,0.454590,0.535687,-0.282236,0.739319,1.716824,-4.333238,-2.125900,-0.706157,-1.725016,-1.923436,-2.934320,-2.623049
2,-1.866406,2.123016,4.026019,2.999926,3.391423,-1.987179,-2.315236,2.869895,-1.496577,-0.023148,0.577370,0.489784,-0.462795,0.038856,0.667739,-4.333238,-2.105031,-0.712914,-1.704633,-1.872910,-2.849818,-2.559951
3,-1.840653,2.021477,3.811492,1.273293,3.135979,-1.959598,-2.278333,2.773628,-1.496879,-0.755514,1.002803,0.187639,-0.910888,-0.700373,-0.044993,-4.152822,-2.100075,-0.719672,-1.684250,-1.822384,-2.765317,-2.496853
4,-1.814899,1.887113,3.567576,2.045407,2.955412,-1.932017,-2.241431,2.977624,-1.498597,-1.085703,1.172006,0.220940,-1.479870,-1.463202,-0.740685,-3.720662,-2.080078,-0.726429,-1.663867,-1.771858,-2.680816,-2.433756
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157,1.399834,-0.214006,-0.607704,0.043807,-1.233012,1.031384,0.819304,-0.883659,0.832198,1.592778,-0.157724,1.208478,1.533417,1.082584,-0.239523,-0.864708,0.050588,1.519517,1.163052,-0.960847,-0.093961,0.627995
158,1.402752,-0.434717,-0.620523,0.210856,-1.299304,1.042716,0.808136,-0.950771,0.824048,0.921228,0.418547,0.303519,0.611670,0.561310,0.072348,-0.835480,0.042723,1.476375,1.168317,-0.968515,-0.074705,0.638908
159,1.405671,-0.524091,-0.631530,0.107070,-1.248662,1.054049,0.796969,-1.028465,0.815727,0.021265,0.873522,-0.046361,-0.155016,-0.230685,-0.276998,-0.847441,0.029152,1.433232,1.173583,-0.976183,-0.055448,0.649820
160,1.408589,-0.614500,-0.640320,0.392942,-1.068274,1.065381,0.785801,-1.103668,0.810089,-0.134082,1.267294,-0.333524,-0.915473,-0.927276,-0.460208,-0.705967,-0.007439,1.390090,1.178848,-0.983851,-0.036192,0.660733


In [120]:
# Alvo para treinamento
train_target = target_data.iloc[start_index:train_split]
train_target

0      249.680
1      219.561
2      304.641
3      262.564
4      299.209
        ...   
157    338.767
158    419.962
159    387.800
160    391.122
161    409.739
Name: Minas Gerais - Consumo de Cimento (t), Length: 162, dtype: float64

In [121]:
def create_batches(t_input, t_target, window_size, start_from):
    
    X_batches = []
    y_batches = []

    train_input_values = t_input.values 

    for i in range(len(t_input) - window_size):
        
        X_window = train_input_values[i:i+window_size, :]
        y_target = t_target[start_from+i+window_size]

        X_batches.append(X_window)
        y_batches.append(y_target)

    return np.array(X_batches), np.array(y_batches)

In [122]:
reshaped_train, reshaped_target = create_batches(train_input, 
                                                 train_target, 
                                                 window_size, 
                                                 start_index)
reshaped_train.shape

(126, 36, 22)

In [123]:
# input para treinamento
test_input = input_data.iloc[train_split - window_size:split_index + 1]
test_input

,Minas Gerais - IDH,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Minas Gerais - value,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Minas Gerais - Produção de Cimento (t),Minas Gerais - Desemprego,Minas Gerais - PIB - Estadual,Minas Gerais - PIB - Construção Civil,Minas Gerais - PIB - Per Capita,Minas Gerais - PIB - Preços de Mercado
126,0.544198,0.888984,-0.460555,-1.048761,-0.368821,0.651397,0.944085,0.045243,1.135116,-1.182549,1.321588,-1.359761,-1.373459,-0.725366,0.710973,0.211800,0.964210,1.008346,0.811298,0.287865,0.502913,0.863809
127,0.559144,0.954254,-0.440372,-0.649312,-0.328087,0.664707,0.947319,0.061828,1.150078,-0.859371,0.729188,-0.670828,-1.055265,-0.689920,0.267703,0.322170,0.924638,1.051039,0.826912,0.237130,0.464014,0.848131
128,0.574090,1.045217,-0.419247,-0.535369,-0.176031,0.678017,0.950553,0.046225,1.162632,-0.827146,-0.202131,-0.538745,-0.592964,-0.218310,0.445679,0.289697,0.882104,1.093731,0.842525,0.186395,0.425115,0.832454
129,0.589036,1.176395,-0.397019,-1.288807,-0.113037,0.691327,0.953786,0.032522,1.166985,0.041639,-0.703217,0.633040,0.045689,-0.008282,-0.092241,0.093304,0.846537,1.136424,0.858138,0.135660,0.386216,0.816777
130,0.603982,1.303259,-0.376532,-1.507550,-0.022703,0.704637,0.957020,0.042757,1.145581,-0.157354,-1.071388,1.646744,0.753868,0.307139,-0.528054,-0.104626,0.796933,1.179116,0.873751,0.084924,0.347317,0.801100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,1.315321,-2.010387,-0.572934,-1.311366,0.589021,1.365911,0.389193,-1.749976,0.874645,0.970821,0.258320,-0.805508,-0.875907,-0.534471,0.236522,-0.587203,0.774558,0.789659,1.149269,-0.772657,0.183801,0.694771
188,1.308842,-1.870713,-0.588777,-1.417143,1.043728,1.376610,0.370392,-1.593005,0.890157,1.225461,-0.397241,-0.305413,-0.235972,0.004219,0.306158,-0.620933,0.806353,0.784605,1.140354,-0.757425,0.173158,0.684584
189,1.302363,-1.806230,-0.612606,-1.597784,1.387010,1.387308,0.351592,-1.351489,0.906034,1.709580,-0.508089,0.370290,0.502599,0.515241,0.206625,-0.975326,0.844284,0.779552,1.131439,-0.742194,0.162516,0.674397
190,1.295885,-1.727496,-0.640956,-1.400787,1.815728,1.398006,0.332791,-1.198492,0.919415,1.790915,-0.713258,0.845625,1.058426,0.868770,0.089936,-1.671516,0.884141,0.774498,1.122523,-0.726963,0.151874,0.664209


In [124]:
reshaped_test, reshaped_test_target = create_batches(test_input, 
                                                     target_data, 
                                                     window_size, 
                                                     train_split - window_size)
reshaped_test.shape 

(30, 36, 22)

In [125]:
def validation_splitter(arr, div_factor, add_factor=0):
    split_factor = len(arr) // div_factor
    positions_to_drop_index = []
    positions_to_drop = []
    for i in range(split_factor):
        pos = len(arr) - (i * div_factor + 1)
        positions_to_drop_index.append(pos)
        positions_to_drop.append(pos + add_factor)
    
    arr_droped = arr[positions_to_drop]
    arr_result = np.delete(arr, positions_to_drop_index, axis=0)
    
    return arr_result, arr_droped

In [126]:
# Rede Neural Recorrente com optmizador Estocástico
def lstm_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                      patience=500, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True,
                                                      start_from_epoch=500)
    train, train_val = validation_splitter(train_input, 7)
    target,target_val = validation_splitter(train_target, 7)
#     display(train.shape)
#     display(train_val.shape)
#     display(target.shape)
#     display(target_val.shape)
    # Método estocástico e learning rate=0.005
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.005)
    model = tf.keras.Sequential([
        tf.keras.layers.LSTM(36, activation='tanh', 
                             return_sequences=True, 
                             input_shape=(reshaped_train.shape[1],
                                          reshaped_train.shape[2])),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.LSTM(180,activation='tanh'),
        tf.keras.layers.Dense(1)
    ])
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train, 
                        target, 
                        epochs=10000,
                        validation_data=(train_val,
                                         target_val),
                        callbacks=[early_stopping], 
                        verbose=want_verbose)
    return model, history

In [127]:
def get_a_good_seed(train_input, train_target, test_input, test_target):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(5)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = lstm_model(train_input, train_target, want_verbose=0, seed=seed)
        current_loss = min(history.history['val_loss'][500:])
        print(f"val_loss: {current_loss}")

        if current_loss < best_loss:
            best_loss = current_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")
            if winner_seed == 0.0:
                return winner_seed

    return winner_seed

In [128]:
winner_seed = get_a_good_seed(reshaped_train, reshaped_target, reshaped_test, reshaped_test_target)
print(f"\n\nfinal_seed: {winner_seed}")

[2868479736, 1051038978, 3570470028, 1838716067, 3768960993]


Step: 0 ___________________________________________
val_loss: 1404.867919921875
winner_seed: 2868479736


Step: 1 ___________________________________________
val_loss: 1341.8397216796875
winner_seed: 1051038978


Step: 2 ___________________________________________
val_loss: 3816.0283203125


Step: 3 ___________________________________________
val_loss: 1201.820068359375
winner_seed: 1838716067


Step: 4 ___________________________________________
val_loss: 1661.61083984375


final_seed: 1838716067


In [129]:
trained_model, history = lstm_model(reshaped_train, 
                                    reshaped_target, 
                                    want_verbose=1, 
                                    seed=winner_seed)

Epoch 1/10000
4/4 [==============================] - 2s 142ms/step - loss: 201654.6875 - val_loss: 7622.4160
Epoch 2/10000
4/4 [==============================] - 0s 11ms/step - loss: 9275.0742 - val_loss: 7644.9062
Epoch 3/10000
4/4 [==============================] - 0s 11ms/step - loss: 7802.2212 - val_loss: 7691.5903
Epoch 4/10000
4/4 [==============================] - 0s 10ms/step - loss: 7663.4785 - val_loss: 7620.7988
Epoch 5/10000
4/4 [==============================] - 0s 11ms/step - loss: 8810.9854 - val_loss: 7817.4238
Epoch 6/10000
4/4 [==============================] - 0s 11ms/step - loss: 7953.1973 - val_loss: 7627.1416
Epoch 7/10000
4/4 [==============================] - 0s 11ms/step - loss: 8191.7666 - val_loss: 10297.2383
Epoch 8/10000
4/4 [==============================] - 0s 11ms/step - loss: 9326.8564 - val_loss: 7752.1006
Epoch 9/10000
4/4 [==============================] - 0s 12ms/step - loss: 7643.9595 - val_loss: 7790.2612
Epoch 10/10000
4/4 [======================

4/4 [==============================] - 0s 10ms/step - loss: 3873.4324 - val_loss: 5572.5879
Epoch 78/10000
4/4 [==============================] - 0s 11ms/step - loss: 3903.3474 - val_loss: 3354.4668
Epoch 79/10000
4/4 [==============================] - 0s 11ms/step - loss: 3277.2166 - val_loss: 3822.9756
Epoch 80/10000
4/4 [==============================] - 0s 10ms/step - loss: 3277.3315 - val_loss: 3068.3113
Epoch 81/10000
4/4 [==============================] - 0s 9ms/step - loss: 2778.6089 - val_loss: 2688.8447
Epoch 82/10000
4/4 [==============================] - 0s 10ms/step - loss: 2991.0078 - val_loss: 2659.7961
Epoch 83/10000
4/4 [==============================] - 0s 10ms/step - loss: 3446.6099 - val_loss: 2919.1257
Epoch 84/10000
4/4 [==============================] - 0s 9ms/step - loss: 3084.2402 - val_loss: 2853.5371
Epoch 85/10000
4/4 [==============================] - 0s 10ms/step - loss: 2490.8572 - val_loss: 3184.0225
Epoch 86/10000
4/4 [==============================] - 

4/4 [==============================] - 0s 10ms/step - loss: 1887.9821 - val_loss: 2543.9861
Epoch 154/10000
4/4 [==============================] - 0s 10ms/step - loss: 2459.4536 - val_loss: 2833.9163
Epoch 155/10000
4/4 [==============================] - 0s 10ms/step - loss: 2323.7000 - val_loss: 2384.1497
Epoch 156/10000
4/4 [==============================] - 0s 9ms/step - loss: 2204.8628 - val_loss: 2353.8916
Epoch 157/10000
4/4 [==============================] - 0s 11ms/step - loss: 1877.0389 - val_loss: 2045.0525
Epoch 158/10000
4/4 [==============================] - 0s 10ms/step - loss: 1727.3362 - val_loss: 2256.6084
Epoch 159/10000
4/4 [==============================] - 0s 10ms/step - loss: 2060.6379 - val_loss: 2912.4453
Epoch 160/10000
4/4 [==============================] - 0s 10ms/step - loss: 2117.7314 - val_loss: 2235.4456
Epoch 161/10000
4/4 [==============================] - 0s 11ms/step - loss: 2117.3767 - val_loss: 2424.8955
Epoch 162/10000
4/4 [========================

4/4 [==============================] - 0s 10ms/step - loss: 1462.0374 - val_loss: 2158.0920
Epoch 230/10000
4/4 [==============================] - 0s 11ms/step - loss: 1983.3303 - val_loss: 2136.6318
Epoch 231/10000
4/4 [==============================] - 0s 10ms/step - loss: 1544.6279 - val_loss: 2099.4319
Epoch 232/10000
4/4 [==============================] - 0s 11ms/step - loss: 2118.5117 - val_loss: 1504.6765
Epoch 233/10000
4/4 [==============================] - 0s 11ms/step - loss: 1659.3818 - val_loss: 1718.3706
Epoch 234/10000
4/4 [==============================] - 0s 11ms/step - loss: 1841.0033 - val_loss: 2010.7053
Epoch 235/10000
4/4 [==============================] - 0s 10ms/step - loss: 1626.0057 - val_loss: 1507.3499
Epoch 236/10000
4/4 [==============================] - 0s 11ms/step - loss: 1567.4619 - val_loss: 1876.4425
Epoch 237/10000
4/4 [==============================] - 0s 11ms/step - loss: 1673.0452 - val_loss: 1719.2109
Epoch 238/10000
4/4 [=======================

4/4 [==============================] - 0s 10ms/step - loss: 1944.4803 - val_loss: 2468.0686
Epoch 306/10000
4/4 [==============================] - 0s 10ms/step - loss: 1606.0314 - val_loss: 2407.6614
Epoch 307/10000
4/4 [==============================] - 0s 11ms/step - loss: 1835.3157 - val_loss: 2560.2627
Epoch 308/10000
4/4 [==============================] - 0s 10ms/step - loss: 1765.9236 - val_loss: 2447.4099
Epoch 309/10000
4/4 [==============================] - 0s 11ms/step - loss: 1385.6583 - val_loss: 2515.5415
Epoch 310/10000
4/4 [==============================] - 0s 11ms/step - loss: 1381.8086 - val_loss: 2369.1187
Epoch 311/10000
4/4 [==============================] - 0s 11ms/step - loss: 1573.2506 - val_loss: 2287.5598
Epoch 312/10000
4/4 [==============================] - 0s 11ms/step - loss: 1746.2773 - val_loss: 1928.4099
Epoch 313/10000
4/4 [==============================] - 0s 10ms/step - loss: 2432.5049 - val_loss: 4220.7476
Epoch 314/10000
4/4 [=======================

...

4/4 [==============================] - 0s 11ms/step - loss: 1692.3822 - val_loss: 2232.5278
Epoch 686/10000
4/4 [==============================] - 0s 12ms/step - loss: 1704.6409 - val_loss: 2532.0044
Epoch 687/10000
4/4 [==============================] - 0s 13ms/step - loss: 2106.7205 - val_loss: 2935.4033
Epoch 688/10000
4/4 [==============================] - 0s 12ms/step - loss: 1616.1367 - val_loss: 3256.7312
Epoch 689/10000
4/4 [==============================] - 0s 12ms/step - loss: 1851.7438 - val_loss: 2617.6096
Epoch 690/10000
4/4 [==============================] - 0s 14ms/step - loss: 1606.3883 - val_loss: 2691.2925
Epoch 691/10000
4/4 [==============================] - 0s 12ms/step - loss: 1483.5303 - val_loss: 3150.0676
Epoch 692/10000
4/4 [==============================] - 0s 12ms/step - loss: 1790.8492 - val_loss: 2321.3669
Epoch 693/10000
4/4 [==============================] - 0s 12ms/step - loss: 1678.9716 - val_loss: 2268.1514
Epoch 694/10000
4/4 [=======================

4/4 [==============================] - 0s 9ms/step - loss: 1537.6830 - val_loss: 2553.6052
Epoch 762/10000
4/4 [==============================] - 0s 10ms/step - loss: 1545.3721 - val_loss: 2523.8564
Epoch 763/10000
4/4 [==============================] - 0s 11ms/step - loss: 1394.3539 - val_loss: 2632.0974
Epoch 764/10000
4/4 [==============================] - 0s 10ms/step - loss: 1994.1105 - val_loss: 2859.5139
Epoch 765/10000
4/4 [==============================] - 0s 11ms/step - loss: 1873.6968 - val_loss: 3001.6191
Epoch 766/10000
4/4 [==============================] - 0s 11ms/step - loss: 1316.1260 - val_loss: 3114.6091
Epoch 767/10000
4/4 [==============================] - 0s 10ms/step - loss: 1741.3568 - val_loss: 2467.2219
Epoch 768/10000
4/4 [==============================] - 0s 10ms/step - loss: 1649.7052 - val_loss: 2986.8723
Epoch 769/10000
4/4 [==============================] - 0s 10ms/step - loss: 1749.3237 - val_loss: 2327.6460
Epoch 770/10000
4/4 [========================

4/4 [==============================] - 0s 16ms/step - loss: 1539.6492 - val_loss: 3005.3955
Epoch 838/10000
4/4 [==============================] - 0s 11ms/step - loss: 1676.4392 - val_loss: 1825.4238
Epoch 839/10000
4/4 [==============================] - 0s 11ms/step - loss: 1550.6846 - val_loss: 2039.7832
Epoch 840/10000
4/4 [==============================] - 0s 12ms/step - loss: 1842.3357 - val_loss: 3228.3811
Epoch 841/10000
4/4 [==============================] - 0s 12ms/step - loss: 1823.8015 - val_loss: 2648.7957
Epoch 842/10000
4/4 [==============================] - 0s 13ms/step - loss: 1757.7185 - val_loss: 2162.6147
Epoch 843/10000
4/4 [==============================] - 0s 12ms/step - loss: 1944.6698 - val_loss: 2385.1311
Epoch 844/10000
4/4 [==============================] - 0s 18ms/step - loss: 1660.5233 - val_loss: 2373.7588
Epoch 845/10000
4/4 [==============================] - 0s 13ms/step - loss: 1909.8826 - val_loss: 2625.4309
Epoch 846/10000
4/4 [=======================

4/4 [==============================] - 0s 11ms/step - loss: 1638.8765 - val_loss: 2340.7285
Epoch 914/10000
4/4 [==============================] - 0s 12ms/step - loss: 1469.3884 - val_loss: 2132.9277
Epoch 915/10000
4/4 [==============================] - 0s 11ms/step - loss: 1333.4633 - val_loss: 2383.9587
Epoch 916/10000
4/4 [==============================] - 0s 11ms/step - loss: 1526.5107 - val_loss: 2216.1572
Epoch 917/10000
4/4 [==============================] - 0s 11ms/step - loss: 1447.9576 - val_loss: 2147.9304
Epoch 918/10000
4/4 [==============================] - 0s 12ms/step - loss: 1600.8292 - val_loss: 2782.5830
Epoch 919/10000
4/4 [==============================] - 0s 12ms/step - loss: 1504.9382 - val_loss: 2572.8550
Epoch 920/10000
4/4 [==============================] - 0s 10ms/step - loss: 1717.9969 - val_loss: 2482.1428
Epoch 921/10000
4/4 [==============================] - 0s 11ms/step - loss: 1610.2625 - val_loss: 2600.1091
Epoch 922/10000
4/4 [=======================

4/4 [==============================] - 0s 11ms/step - loss: 1307.9349 - val_loss: 1791.5854
Epoch 990/10000
4/4 [==============================] - 0s 11ms/step - loss: 1180.5436 - val_loss: 1543.5321
Epoch 991/10000
4/4 [==============================] - 0s 12ms/step - loss: 1357.0625 - val_loss: 1798.2729
Epoch 992/10000
4/4 [==============================] - 0s 11ms/step - loss: 1211.5862 - val_loss: 1690.4609
Epoch 993/10000
4/4 [==============================] - 0s 10ms/step - loss: 1006.5913 - val_loss: 1657.3793
Epoch 994/10000
4/4 [==============================] - 0s 11ms/step - loss: 1271.9749 - val_loss: 1470.8247
Epoch 995/10000
4/4 [==============================] - 0s 11ms/step - loss: 1138.4900 - val_loss: 1375.7683
Epoch 996/10000
4/4 [==============================] - 0s 11ms/step - loss: 1201.0984 - val_loss: 1960.9462
Epoch 997/10000
4/4 [==============================] - 0s 10ms/step - loss: 1385.9678 - val_loss: 1454.5955
Epoch 998/10000
4/4 [=======================

In [130]:
def mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    results_data = []

    for i in range(len(test_target)):
        prediction = model.predict(test_input[i:i+1])
        target = test_target[start_target + i]
        error = np.abs(prediction - target)
        errors.append(error)
        error_percent.append(error/target)
        results_data.append([f"Month-{i + 1}", 
                             prediction[0][0], 
                             target, 
                             error[0][0]])

    df_results = pd.DataFrame(results_data, columns=["Month", "Prediction", "Target", "Error"])

    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return df_results, mae, mape

In [131]:
df_results, mae, mape = mae_mape_calculator(trained_model, 
                                            reshaped_test, 
                                            reshaped_test_target, 
                                            start_index)
pd.set_option('display.max_columns', None)
df_results.T

1/1 [==============================] - 0s 16ms/step


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29
Month,Month-1,Month-2,Month-3,Month-4,Month-5,Month-6,Month-7,Month-8,Month-9,Month-10,Month-11,Month-12,Month-13,Month-14,Month-15,Month-16,Month-17,Month-18,Month-19,Month-20,Month-21,Month-22,Month-23,Month-24,Month-25,Month-26,Month-27,Month-28,Month-29,Month-30
Prediction,406.830292,424.267059,410.155365,417.764801,414.851898,424.84671,426.888031,412.505127,391.066498,365.6987,411.455566,406.150879,392.108643,394.837128,408.34024,415.380371,410.179688,411.088013,399.537323,475.054718,495.675629,554.662476,512.059326,510.592346,514.822754,530.88147,529.336548,520.561035,539.350586,539.427734
Target,413.46,497.019,476.478,421.395,413.641,338.033,431.028,305.105,402.406,419.419,336.054,568.395,541.488,537.851,526.265,559.293,466.646,416.049,501.489,460.699,445.583,509.271,532.602,500.905,588.495,589.349,543.982,614.074,557.419,412.582
Error,6.6297,72.751953,66.322632,3.630188,1.210907,86.813721,4.139984,107.400116,11.339508,53.720306,75.401581,162.244141,149.379333,143.013885,117.924774,143.912659,56.466309,4.960999,101.951691,14.355713,50.092621,45.391479,20.542664,9.687347,73.672241,58.467529,14.645447,93.512939,18.06842,126.845734


In [132]:
display(mae)
display(mape)

63.14988

0.13303748

In [133]:
def year_mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    
    target_sum = 0
    prediction_sum = 0
    
    for i in range(len(test_target)):
        if i % 12 == 0 and i != 0:
            error = np.abs(target_sum - prediction_sum)
            errors.append(error)
            error_percent.append(error / target_sum)
            print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}; MAPE:{abs(prediction_sum - target_sum)/target_sum}")
            target_sum = 0
            prediction_sum = 0
            
        prediction = model.predict(test_input[i:i+1])
        target_sum += test_target[start_target + i]
        prediction_sum += prediction
        
    error = np.abs(target_sum - prediction_sum)
    errors.append(error)
    error_percent.append(error / target_sum)
    print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}; MAPE:{abs(prediction_sum - target_sum)/target_sum}")
        
    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return errors, mae, mape

In [134]:
errors, mae, mape = year_mae_mape_calculator(trained_model, 
                                             reshaped_test, 
                                             reshaped_test_target, 
                                             start_index)
display(errors)
display(mae)
display(mape)

1/1 [==============================] - 0s 17ms/step
Ano-0: |Prediction[[4912.481]] - Target[5022.433000000001]| =  Error: [[109.95215]]; MAPE:[[0.02189221]]
1/1 [==============================] - 0s 16ms/step
Ano-0: |Prediction[[5379.516]] - Target[5998.141]| =  Error: [[618.625]]; MAPE:[[0.10313612]]
1/1 [==============================] - 0s 15ms/step
Ano-5: |Prediction[[3174.3801]] - Target[3305.901]| =  Error: [[131.52075]]; MAPE:[[0.03978363]]


[array([[109.95215]], dtype=float32),
 array([[618.625]], dtype=float32),
 array([[131.52075]], dtype=float32)]

286.6993

0.05493732